# Naive Bayes

## Lectura de datos

In [2]:
import pandas as pd
import nltk
import gensim
import sqlite3

sqlite_db = "../dataset/fake_news.sqlite"

conn = sqlite3.connect(sqlite_db)
df = pd.read_sql_query("SELECT * FROM NEWS", conn)
conn.close()

## Total de palabras en el dataset

In [3]:
list_of_words = []
unique_words = set()
for document in df_news.clean_joined:
    for word in gensim.utils.simple_preprocess(document):
        list_of_words.append(word)
        unique_words.add(word)
        
total_words = len(list_of_words)  # total words
unique_words = len(unique_words)   # total unique words
print("Total words:" + str(total_words) + " unique_words:" + str(unique_words))

Total words:8844981 unique_words:96148


## División de los datos

In [4]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.clean_joined, df.isfake, test_size = 0.2)

In [5]:
from nltk import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words = total_words)
tokenizer.fit_on_texts(x_train)
x_train_seq = tokenizer.texts_to_sequences(x_train)
x_test_seq = tokenizer.texts_to_sequences(x_test)

In [6]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

x_train_padded = pad_sequences(x_train_seq, maxlen = 40, padding = 'post', truncating = 'post')
x_test_padded = pad_sequences(x_test_seq, maxlen = 40, truncating = 'post') 

In [7]:
print(x_train_seq[0])

[7179, 1642, 398, 629, 1453, 391, 1201, 4780, 1238, 21, 3393, 564, 1453, 96, 2262, 161, 21, 1321, 3201, 7039, 3864, 2096, 1008, 2411, 34, 229, 21, 3393, 629, 564, 7179, 1642, 2411, 1008, 7135, 409, 538, 1238, 1201, 2496, 539, 2411, 3469, 7179, 1642, 399, 564, 9782, 21, 2721, 1647, 856, 1300, 1008, 740, 1756, 56, 3201, 7039, 3864, 22685, 468, 10266, 250, 1238, 983, 812, 2096, 1289, 3655, 2953, 571, 1297, 1008, 3864, 2496, 539, 1642, 7, 1008, 7135, 22, 4780, 67, 1642, 10680, 3864, 104, 689, 42, 1008, 7135, 2496, 539, 1642, 751, 302, 1279, 3030, 423, 99, 194, 6278, 689, 42, 54, 3099, 1480, 296, 689, 42, 1008, 7135, 1625, 1642, 67, 7786, 595, 16, 5, 90, 31, 69, 4, 2678, 1922, 13073, 996, 2839, 477, 678, 2056, 3864, 1642, 67, 229, 1049, 407, 564, 1201, 2953, 564, 564, 806, 3363, 2096, 9451, 812, 3864, 539, 1238, 3864, 1008, 7135, 250, 3864, 1325, 28, 173, 6764, 338, 2221, 2302, 161, 3864, 891, 2953, 842, 5401, 3201, 7039, 3864, 5, 247, 376, 176, 1241, 1086, 1349, 629, 1773, 34, 4089, 173, 3

In [8]:
print(x_train_padded[0])

[7179 1642  398  629 1453  391 1201 4780 1238   21 3393  564 1453   96
 2262  161   21 1321 3201 7039 3864 2096 1008 2411   34  229   21 3393
  629  564 7179 1642 2411 1008 7135  409  538 1238 1201 2496]


In [9]:
import numpy as np
y_train = np.asarray(y_train).astype('float32').reshape((-1,1))

In [10]:
x_train_padded

array([[ 7179,  1642,   398, ...,  1238,  1201,  2496],
       [ 4740,     1,   656, ...,   636,   381,    80],
       [  733,   473,  1532, ...,    21,    33,   771],
       ...,
       [ 2904,   200,  5902, ...,    72,   696,  2483],
       [ 3696,  2566, 20230, ...,     0,     0,     0],
       [  204,   331,  3030, ...,     0,     0,     0]], dtype=int32)

## Construcción del modelo

In [11]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
%time clf.fit(x_train_padded, y_train)

CPU times: user 16.8 ms, sys: 0 ns, total: 16.8 ms
Wall time: 16.5 ms


/home/drojasc@EJC-CAIMI-NAL.MIL/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


GaussianNB()

In [12]:
y_pred=clf.predict(x_test_padded)

In [13]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.5910913140311804
